In [1]:
import os

In [2]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\Data Science Projects\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\Data Science Projects\\Text-Summarization'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories


In [50]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [44]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size
import shutil

In [51]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [52]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-21 02:16:54,656: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-21 02:16:54,672: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-21 02:16:54,672: INFO: common: created directory at: artifacts]
[2023-10-21 02:16:54,680: INFO: common: created directory at: artifacts/data_ingestion]


[2023-10-21 02:17:05,537: INFO: 3078253932: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 702C:5C4F8:138EB:19D66:6532E73E
Accept-Ranges: bytes
Date: Fri, 20 Oct 2023 20:46:57 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4742-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1697834816.243128,VS0,VE1416
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: d08a551652c68078852b6c13b0d5256a5135c0d7
Expires: Fri, 20 Oct 2023 20:51:57 GMT
Source-Age: 1

]


EOFError: 